In [1]:
%load_ext autoreload
%autoreload 2

import os
import yaml
import time
import torch
import random
import numpy as np

from torch.utils import data

from tensorboardX import SummaryWriter

import data_loader
import models
from optimizer import get_optimizer
from loss import cross_entropy_2d.CrossEntropy2d

SEED = 1337

def train(config, writer=None):
    # setup seeds
    torch.manual_seed(config["seed"])
    torch.cuda.manual_seed(config["seed"])
    np.random.seed(config["seed"])

    # setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # setup augmentations
    ## ah todo
    
    # setup dataloader
    mit_data = data_loader.get_loader("mit_sceneparsing_benchmark")
    train_data_raw = mit_data("training")
    train_data = data.DataLoader(train_data_raw, batch_size=config["batch_sz"],\
                                 num_workers=1, shuffle=True)
    
    val = data.DataLoader(mit_data("validation"), batch_size=config["batch_sz"],\
                                 num_workers=1)

    # setup metrics
    ## ah todo
    
    # setup model
    model = models.get_model("fcn32s", train_data_raw.n_classes).to(device)
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    
    # setup optimizer
    optimizer = get_optimizer("adam")(model.parameters())
    
    print(f"Optimizer: {optimizer}")
    
    # setup learning rate scheduler (optim.lr_scheduler)
    ## ah todo
    loss_fn = CrossEntropy2d()
        
    i = 0
    while i < config["epochs"]:
        i += 1
        for (images, labels) in train_data:
            start_ts = time.time()
            model.train()
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            out = model(images)
            
            loss = loss_fn(out, labels)
            
            loss.backward()
            optimizer.step()
            
            print(f"Batch Loss : {loss} ... time : {time.time() - start_ts}")
            
            
config = {"batch_sz": 256, "epochs": 2, "seed": 42}

train(config)

ImportError: cannot import name 'CrossEntropy2d' from 'loss' (/home/ammar/src/torch-sandbox/vision/semantic-segmentation/loss/__init__.py)